### 1. Load libraries, set tickers and dates

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import yfinance as yf
from scipy.optimize import minimize

/Users/seanhicks/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [2]:
start = dt.datetime(2015,1,1) 
end = dt.datetime.now()

In [3]:
tickers = ['VB', 'VO', 'VV', 'VNQ', 'VEA', 'VWO'] #list securities/funds in portfolio

### 2. Download price data, convert to returns

In [4]:
df = yf.download(tickers, start, end) 

[*********************100%***********************]  6 of 6 completed


In [5]:
df

Adj Close                                                \
                    VB        VEA        VNQ          VO          VV   
Date                                                                   
2015-01-02  103.457245  29.794954  59.919506  109.435677   81.582298   
2015-01-05  101.864632  29.130909  60.247463  107.505219   80.180939   
2015-01-06  100.565643  28.806791  60.845058  106.336304   79.402390   
2015-01-07  101.775681  29.115112  61.777870  107.691162   80.302040   
2015-01-08  103.394981  29.557793  62.011063  109.621658   81.815857   
...                ...        ...        ...         ...         ...   
2022-12-16  183.359634  41.750000  82.918053  203.996216  174.722443   
2022-12-19  180.972656  41.660000  81.862968  201.847946  172.930847   
2022-12-20  181.808105  41.860001  81.655891  202.215942  173.348877   
2022-12-21  184.940994  42.279999  82.355995  205.319000  175.887009   
2022-12-22  182.660004  41.910000  82.010002  202.759995  173.449997   

                            Close                                    ...  \
                  VWO          VB        VEA        VNQ          VO  ...   
Date                                                                 ...   
2015-01-02  31.521152  116.279999  37.689999  82.220001  123.580002  ...   
2015-01-05  31.019161  114.489998  36.849998  82.669998  121.400002  ...   
2015-01-06  30.867788  113.029999  36.439999  83.489998  120.080002  ...   
2015-01-07  31.576927  114.389999  36.830002  84.769997  121.610001  ...   
2015-01-08  32.110771  116.209999  37.389999  85.089996  123.790001  ...   
...               ...         ...        ...        ...         ...  ...   
2022-12-16  38.875000  184.360001  42.240002  84.089996  205.110001  ...   
2022-12-19  38.950001  181.960007  41.660000  83.019997  202.949997  ...   
2022-12-20  38.820000  182.800003  41.860001  82.809998  203.320007  ...   
2022-12-21  39.180000  185.949997  42.279999  83.519997  206.440002  ...   
2022-12-22  38.830002  182.660004  41.910000  82.010002  202.759995  ...   

                 Open                                      Volume            \
                  VNQ          VO          VV        VWO       VB       VEA   
Date                                                                          
2015-01-02  81.699997  123.930000   94.800003  39.950001   622800   5970800   
2015-01-05  82.000000  122.900002   93.739998  39.419998   596400   8787700   
2015-01-06  82.750000  121.669998   92.849998  38.930000   569400   5413000   
2015-01-07  83.809998  121.080002   92.510002  39.450001   503400   4906100   
2015-01-08  85.349998  122.660004   93.660004  40.160000   406600   5459800   
...               ...         ...         ...        ...      ...       ...   
2022-12-16  85.419998  205.779999  176.440002  39.680000  1407200  14511200   
2022-12-19  84.000000  205.169998  175.910004  39.160000   791700  17846500   
2022-12-20  82.629997  202.669998  173.460007  38.750000   834300  15525100   
2022-12-21  83.360001  204.860001  175.419998  38.799999   728800  14202600   
2022-12-22  81.669998  203.649994  174.410004  39.070000  1201000  17651300   

                                                
                VNQ      VO       VV       VWO  
Date                                            
2015-01-02  5570500  452100  1211500  17844900  
2015-01-05  6073700  545600   486200  17336400  
2015-01-06  7577100  452600   288500  13779200  
2015-01-07  6920700  493800   229500  11570800  
2015-01-08  5255000  653300   214400   9996800  
...             ...     ...      ...       ...  
2022-12-16  8166400  825200   536800  12156700  
2022-12-19  3776100  816200   352400  19755100  
2022-12-20  6614800  802600   443600  18051400  
2022-12-21  7101700  874300   314600  15728300  
2022-12-22  6619400  902900   513200  17285500  

[2009 rows x 36 columns]

In [6]:
prices = df[['Adj Close']]
prices

Adj Close                                                \
                    VB        VEA        VNQ          VO          VV   
Date                                                                   
2015-01-02  103.457245  29.794954  59.919506  109.435677   81.582298   
2015-01-05  101.864632  29.130909  60.247463  107.505219   80.180939   
2015-01-06  100.565643  28.806791  60.845058  106.336304   79.402390   
2015-01-07  101.775681  29.115112  61.777870  107.691162   80.302040   
2015-01-08  103.394981  29.557793  62.011063  109.621658   81.815857   
...                ...        ...        ...         ...         ...   
2022-12-16  183.359634  41.750000  82.918053  203.996216  174.722443   
2022-12-19  180.972656  41.660000  81.862968  201.847946  172.930847   
2022-12-20  181.808105  41.860001  81.655891  202.215942  173.348877   
2022-12-21  184.940994  42.279999  82.355995  205.319000  175.887009   
2022-12-22  182.660004  41.910000  82.010002  202.759995  173.449997   

                       
                  VWO  
Date                   
2015-01-02  31.521152  
2015-01-05  31.019161  
2015-01-06  30.867788  
2015-01-07  31.576927  
2015-01-08  32.110771  
...               ...  
2022-12-16  38.875000  
2022-12-19  38.950001  
2022-12-20  38.820000  
2022-12-21  39.180000  
2022-12-22  38.830002  

[2009 rows x 6 columns]

In [7]:
returns=prices.pct_change(1).dropna()
returns

Adj Close                                                  
                  VB       VEA       VNQ        VO        VV       VWO
Date                                                                  
2015-01-05 -0.015394 -0.022287  0.005473 -0.017640 -0.017177 -0.015926
2015-01-06 -0.012752 -0.011126  0.009919 -0.010873 -0.009710 -0.004880
2015-01-07  0.012032  0.010703  0.015331  0.012741  0.011330  0.022973
2015-01-08  0.015910  0.015204  0.003775  0.017926  0.018852  0.016906
2015-01-09 -0.008433 -0.004814  0.000470 -0.007189 -0.008353 -0.004963
...              ...       ...       ...       ...       ...       ...
2022-12-16 -0.010094 -0.007752 -0.025834 -0.012565 -0.011599 -0.001264
2022-12-19 -0.013018 -0.002156 -0.012724 -0.010531 -0.010254  0.001929
2022-12-20  0.004616  0.004801 -0.002530  0.001823  0.002417 -0.003338
2022-12-21  0.017232  0.010033  0.008574  0.015345  0.014642  0.009274
2022-12-22 -0.012334 -0.008751 -0.004201 -0.012464 -0.013856 -0.008933

[2008 rows x 6 columns]

### 3. Prepare data for optimization

In [8]:
returns.describe() #review data - emsure all tickers have the same number of observations in the data

Adj Close                                                      \
                VB          VEA          VNQ           VO           VV   
count  2008.000000  2008.000000  2008.000000  2008.000000  2008.000000   
mean      0.000377     0.000236     0.000249     0.000387     0.000447   
std       0.013623     0.011457     0.013500     0.012585     0.011924   
min      -0.130789    -0.111803    -0.177277    -0.123888    -0.119887   
25%      -0.005578    -0.004847    -0.005560    -0.004619    -0.003880   
50%       0.000818     0.000706     0.000776     0.000896     0.000559   
75%       0.007198     0.006097     0.006988     0.006424     0.005923   
max       0.101777     0.088951     0.089967     0.108837     0.097262   

                    
               VWO  
count  2008.000000  
mean      0.000191  
std       0.013190  
min      -0.120891  
25%      -0.006782  
50%       0.000737  
75%       0.007493  
max       0.077822

In [10]:
num_stocks = len(returns.columns) #Create the initial weights for the optimization function 

init_weights = [1/num_stocks] * num_stocks

In [11]:
rf_rate = .03 #set the risk-free rate for Sharpe Ratio calculation 

In [12]:
vcv = returns.cov() #create the variance-co-variance matrix (needed for portfolio variance/std. dev. calcs)

In [13]:
def get_ret_sd_sr(weights): #returns the portfolios expected return, variance, standard deviation, and sharpe ratio in one array
    ret = np.dot(np.transpose(weights), returns.mean()) * 252
    var = np.dot(np.transpose(weights), np.dot(vcv, weights))
    sd = np.sqrt(var) * np.sqrt(252)
    sr = (ret-rf_rate)/sd
    
    return np.array([ret,sd,sr])

In [14]:
get_ret_sd_sr(init_weights)

array([0.07924234, 0.18480704, 0.26645275])

In [15]:
def neg_sharpe(weights): #function that transforms Sharpe Ratio to a negative value since the optimization we're running is a minimization
    return get_ret_sd_sr(weights)[2] * -1

In [16]:
neg_sharpe(init_weights)

-0.26645274949776687

In [17]:
bounds = tuple((0.05,.333333) for i in range (num_stocks)) #setting the bounds for the weights of the funds - adjust as needed

In [18]:
cons = ({'type' : 'eq', 'fun': lambda x : np.sum(x) - 1}) #setting the constraint that the portfolio weights must sum to 100%

### 4. Run optimization 

In [19]:
results = minimize(neg_sharpe, init_weights, bounds=bounds, constraints=cons) #optimization function

In [20]:
optimized_weights = pd.DataFrame(results['x'].round(3)*100) #converts results to table in percent form with one decimal
optimized_weights.index = returns.columns
optimized_weights

0
Adj Close VB   18.3
          VEA   5.0
          VNQ   5.0
          VO   33.3
          VV   33.3
          VWO   5.0

### 5. Review optimial portfolio results 

In [21]:
opt_weights = optimized_weights.values.tolist() #create list of optimized weights 
opt_weights

[[18.3], [5.0], [5.0], [33.300000000000004], [33.300000000000004], [5.0]]

In [22]:
get_ret_sd_sr(opt_weights) #input optimized weights to view optimal portfolio returns, volatility and share ratio

<ipython-input-13-bd38e2a4466a>:7: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array([ret,sd,sr])


array([[9.587089812477119],
       [19.1471287861675],
       [0.4991395795792353]], dtype=object)

In [23]:
exp_returns =returns.mean()*252 #view expected returns, based on historal average, for each security 
exp_returns

Adj Close  VB     0.094904
           VEA    0.059499
           VNQ    0.062658
           VO     0.097488
           VV     0.112674
           VWO    0.048231
dtype: float64